# 5.1 スクレイピング

## 5.1.2 スクレイピング環境の準備

```
(pydataenv) $ pip install requests==2.28.1
(pydataenv) $ pip install beautifulsoup4==4.11.1
```

* Requests: https://requests.readthedocs.io/
* BeautifulSoup4: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

## 5.1.3 Webページをダウンロード

* SEshop.com https://www.seshop.com/

In [1]:
import requests

r = requests.get('https://www.seshop.com/')  # URLにアクセス
print(type(r))
print(r.status_code)  # ステータスコードを確認

<class 'requests.models.Response'>
200


In [2]:
text = r.text  # HTMLのソースコードを取得する
for line in text.split('\n'):
    if'<title>' in line or '<h2>' in line:
        print(line.strip())

<title>SEshop｜ 翔泳社の本・電子書籍通販サイト</title>
<h2>新刊書籍 <span class="pull-right links"><a href="/product/1/"><span class="glyphicon glyphicon-chevron-right"></span> 一覧を見る</a></span></h2>
<h2>あなたへのおすすめ</h2>
<h2>書籍ランキング <span class="pull-right links"><a href="/ranking/1/"><span class="glyphicon glyphicon-chevron-right"></span> その他を見る</a></span></h2>
<h2>電子書籍ランキング<span class="pull-right links"><a href="/ranking/327/"><span class="glyphicon glyphicon-chevron-right"></span> その他を見る</a></span></h2>
<h2>セミナー・イベント <span class="pull-right links"><a href="/product/406/"><span class="glyphicon glyphicon-chevron-right"></span> 一覧を見る</a></span></h2>
<h2>商品カテゴリー</h2>
<h2>人気商品ランキング</h2>
<h2>キャンペーン・特集</h2>
<h2>新着記事</h2>


## 5.1.4 Webページから要素を抜き出す

In [3]:
from bs4 import BeautifulSoup

# HTMLを解析したBeautifulSoupオブジェクトを生成
soup = BeautifulSoup(text, 'html.parser')
print(soup.title)  # <title> タグの情報を取得
print(soup.h2)  # <h2> タグの情報を取得
# h2タグの中のaタグのhref属性
print(soup.h2.a['href'])

<title>SEshop｜ 翔泳社の本・電子書籍通販サイト</title>
<h2>新刊書籍 <span class="pull-right links"><a href="/product/1/"><span class="glyphicon glyphicon-chevron-right"></span> 一覧を見る</a></span></h2>
/product/1/


In [4]:
atags = soup.find_all('a')  # すべてのaタグを取得
print('aタグの数:', len(atags))  # aタグの数を取得
for atag in atags[:5]:
    print('タイトル:', atag.text)  # aタグのテキストを取得
    print('リンク:', atag['href'])  # aタグのリンクを取得

aタグの数: 239
タイトル: 
リンク: /
タイトル:  会員登録
リンク: https://www.seshop.com/regist/
タイトル:  ログイン
リンク: #modalLogin
タイトル:  ヘルプ
リンク: /help
タイトル: 会員登録
リンク: https://www.seshop.com/regist


## 5.1.5 記事の一覧を抜き出す

* Pythonの書籍一覧: https://www.seshop.com/product/616

In [5]:
from datetime import datetime

import requests
from bs4 import BeautifulSoup

r = requests.get("https://www.seshop.com/product/616")
soup = BeautifulSoup(r.text, "html.parser")

books = []  # 各書籍の情報を格納するリスト

# CSSセレクターで <div class="list">の中の<div class="inner"> を取得
divs = soup.select("div.list div.inner")
for div in divs:
    img_url = div.find("img")["src"]
    # 日付の文字列を取得
    day = div.find("span", class_="date").text.strip()
    day = day.replace("発売", "")
    # 日付をdatetimeに変換
    published = datetime.strptime(day, "%Y.%m.%d")
    
    div_txt = div.find("div", class_="txt")
    a_tag = div_txt.find("a")  # aタグを取得
    title = a_tag.text.strip()  # 書籍タイトルを取得
    url = a_tag["href"]  # 書籍URLを取得
    
    # 販売価格を取得
    price_s = div_txt.find("span", class_="sale-price").parent.text
    price_s = price_s.strip()
    price_s = price_s.replace("円（税込）送料無料", "")
    price_s = price_s.replace(",", "")
    price = int(price_s)
    
    book = {
        "title": title,
        "img_url": img_url,
        "url": url,
        "price": price,
        "published": published,
    }
    books.append(book)

In [6]:
books[:3]

[{'title': 'Pythonによるあたらしいデータ分析の教科書 第2版',
  'img_url': '/static/images/product/25331/L.png',
  'url': '/product/detail/25331',
  'price': 2838,
  'published': datetime.datetime(2022, 10, 24, 0, 0)},
 {'title': 'PyTorchで作る！深層学習モデル・AI アプリ開発入門',
  'img_url': '/static/images/product/25063/L.png',
  'url': '/product/detail/25063',
  'price': 3520,
  'published': datetime.datetime(2022, 9, 5, 0, 0)},
 {'title': 'テスト駆動Python 第2版',
  'img_url': '/static/images/product/25262/L.png',
  'url': '/product/detail/25262',
  'price': 3300,
  'published': datetime.datetime(2022, 8, 30, 0, 0)}]

In [7]:
import pandas as pd

df = pd.DataFrame(books)  # 辞書をDataFrameに変換
df.head()

,title,img_url,url,price,published
0,Pythonによるあたらしいデータ分析の教科書 第2版,/static/images/product/25331/L.png,/product/detail/25331,2838,2022-10-24
1,PyTorchで作る！深層学習モデル・AI アプリ開発入門,/static/images/product/25063/L.png,/product/detail/25063,3520,2022-09-05
2,テスト駆動Python 第2版,/static/images/product/25262/L.png,/product/detail/25262,3300,2022-08-30
3,Python1年生 第2版 体験してわかる！会話でまなべる！プログラミングのしくみ,/static/images/product/25005/L.png,/product/detail/25005,2178,2022-08-04
4,Pythonで動かして学ぶ！あたらしい機械学習の教科書 第3版,/static/images/product/25020/L.png,/product/detail/25020,2970,2022-07-19


## 5.1.6 スクレイピングで気をつけること

* Robots.txtの仕様: https://developers.google.com/search/docs/crawling-indexing/robots/intro

## 5.1.7 次のステップ

* Selenium: https://www.seleniumhq.org/
* Scrapy: https://scrapy.org/